<a href="https://colab.research.google.com/github/budennovsk/AuthorBooksComments/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_V_2_kion_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall pyspark -y  # Удалите текущую
!pip install pyspark==3.2.1  # Установите 3.2.1 (проверьте на compatibility с replay)

Found existing installation: pyspark 4.0.2
Uninstalling pyspark-4.0.2:
  Successfully uninstalled pyspark-4.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 14.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853627 sha256=9986c2e1eed9ecefb0b161dab1d1273054623241adaa0aa7afe8f87742c0fc46
  Stored in directory: /root/.cache/pip/wheels/38/88/a5/f4282f48f73210c80f64d427e8515c6cee57b04fa74797add4
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.2 requires pyspark[connect]~=4.0.0, 

In [1]:
!pip install replay-rec==0.21.1 requests==2.32.3

In [2]:
import os
import pickle
import sys

sys.path.append('../..')
import polars as pl
import numpy as np

from dotenv import load_dotenv
from replay.preprocessing.filters import MinCountFilter
from sentence_transformers import SentenceTransformer
load_dotenv()

False

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import requests
# from tqdm.auto import tqdm
# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
# req = requests.get(url, stream=True)

# with open('kion_train.zip', "wb") as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='Downloading the kion dataset...',
#                         total=total_size_in_bytes,
#                         unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

In [ ]:
# !unzip kion_train.zip -x '__MACOSX/*'

In [ ]:
# import pandas as pd
# import requests
# from tqdm.auto import tqdm
# from pprint import pprint

# import warnings

# interactions = (
#     pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])
#     .rename(columns={'total_dur': 'weight',
#                      'last_watch_dt': 'datetime'})
# )
# users = pd.read_csv('data_original/users.csv')
# items = pd.read_csv('data_original/items.csv')

# filter_interactions = interactions[interactions['user_id'].isin(users['user_id'].unique())]
# ## ! если хотите быстро прогнать этот ноутбук - раскомментируйте эти строки - она уменьшает данные
# import numpy as np
# user_ids = np.random.choice(filter_interactions['user_id'].unique(), size=50000, replace=False)
# interactions_final = filter_interactions[filter_interactions['user_id'].isin(user_ids.tolist())]
# ###

# print(interactions_final.shape, interactions_final.user_id.nunique())


In [ ]:
# import pandas as pd
# import os

# # Путь к папке
# path_save = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions'


# # Создание папки, если нет
# os.makedirs(path_save, exist_ok=True)

# # Список DataFrame (замените на ваши реальные переменные)
# dataframes = {
#     'interactions': interactions_final,  # Ваш Pandas DataFrame
#     'users': users,                # Ваш Pandas DataFrame
#     'items': items                 # Ваш Pandas DataFrame
# }

# # Сохранение каждого в Parquet
# for name, df in dataframes.items():
#     path_file = f'{path_save}/{name}.parquet'
#     df.to_parquet(path_file)
#     print(f'Сохранено: {path_file}')

# print("Все файлы сохранены!")

In [84]:
path_interaction = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/interactions.parquet'
path_users = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/users.parquet'
path_items = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/items.parquet'


interactions_orig = pl.read_parquet(path_interaction)
users = pl.read_parquet(path_users)
items = pl.read_parquet(path_items)

print("Shape:", interactions_orig.shape)
print("Колонки:", interactions_orig.columns)
print("Пример данных:")
print(interactions_orig.head(5))

Shape: (298123, 6)
Колонки: ['user_id', 'item_id', 'datetime', 'weight', 'watched_pct', '__index_level_0__']
Пример данных:
shape: (5, 6)
┌─────────┬─────────┬─────────────────────┬────────┬─────────────┬───────────────────┐
│ user_id ┆ item_id ┆ datetime            ┆ weight ┆ watched_pct ┆ __index_level_0__ │
│ ---     ┆ ---     ┆ ---                 ┆ ---    ┆ ---         ┆ ---               │
│ i64     ┆ i64     ┆ datetime[ns]        ┆ i64    ┆ f64         ┆ i64               │
╞═════════╪═════════╪═════════════════════╪════════╪═════════════╪═══════════════════╡
│ 176549  ┆ 9506    ┆ 2021-05-11 00:00:00 ┆ 4250   ┆ 72.0        ┆ 0                 │
│ 743269  ┆ 4398    ┆ 2021-05-04 00:00:00 ┆ 67     ┆ 0.0         ┆ 25                │
│ 841859  ┆ 3935    ┆ 2021-06-15 00:00:00 ┆ 8862   ┆ 100.0       ┆ 27                │
│ 750995  ┆ 11699   ┆ 2021-03-15 00:00:00 ┆ 12998  ┆ 100.0       ┆ 66                │
│ 804607  ┆ 5220    ┆ 2021-07-11 00:00:00 ┆ 171    ┆ 0.0         ┆ 69          

In [86]:
import polars as pl


interactions = interactions_orig.select(
    pl.col('user_id').alias('user_id'),
    pl.col('item_id').alias('item_id'),
    pl.col('watched_pct').alias('rating'),
    pl.col('datetime').alias('timestamp'),
)


# Сортировка по 'datetime' (ascending по умолчанию)
interactions = interactions.sort(pl.col('timestamp'))

interactions

user_id,item_id,rating,timestamp
i64,i64,f64,datetime[ns]
541436,4072,99.0,2021-03-13 00:00:00
914535,8639,100.0,2021-03-13 00:00:00
866260,14703,35.0,2021-03-13 00:00:00
405099,9665,100.0,2021-03-13 00:00:00
110490,8822,100.0,2021-03-13 00:00:00
…,…,…,…
485385,1562,1.0,2021-08-22 00:00:00
866847,1737,62.0,2021-08-22 00:00:00
674573,2220,1.0,2021-08-22 00:00:00


In [ ]:
# # Keep positive interactions only
# MIN_RATING = 3

# interactions = interactions.filter(pl.col('rating') > MIN_RATING).drop('rating')
# len(interactions)

In [87]:
interactions.select(pl.col('user_id', 'item_id').n_unique())

user_id,item_id
u32,u32
50000,8245


In [7]:
# # Split data into train/val and test subsets
# TIME_THRESHOLD_QUANTILE = 0.9
# TIME_THRESHOLD = interactions.get_column('timestamp').quantile(TIME_THRESHOLD_QUANTILE)

# condition = pl.col('timestamp') > TIME_THRESHOLD

# train_val_interactions = interactions.filter(~condition)
# test_interactions = interactions.filter(condition)

# len(train_val_interactions), len(test_interactions)

In [88]:
import polars as pl
from datetime import datetime

# Фиксированный порог
THRESHOLD_STR = "2021-08-10 00:00:00"
THRESHOLD_DT = datetime.fromisoformat(THRESHOLD_STR)


# Фильтруем: тест — все позже порога, train_val — все не позже порога
condition = pl.col("timestamp") > THRESHOLD_DT

train_val_interactions = interactions.filter(~condition)
test_interactions = interactions.filter(condition)

print(len(train_val_interactions), len(test_interactions))
print(len(test_interactions)/len(train_val_interactions))

259535 38588
0.14868129539368485


In [89]:
# Apply N-core filtering to train/val data
N = 3
shape = None

while shape != train_val_interactions.shape:
    shape = train_val_interactions.shape

    train_val_interactions = MinCountFilter(N, groupby_column='user_id').transform(
        train_val_interactions
    )
    train_val_interactions = MinCountFilter(N, groupby_column='item_id').transform(
        train_val_interactions
    )

len(train_val_interactions)

224925

In [90]:
# Assign tail interactions back to test users
test_users = test_interactions.get_column('user_id').unique()
test_interactions = pl.concat(
    [test_interactions, train_val_interactions.filter(pl.col('user_id').is_in(test_users))]
)
len(test_interactions)

/tmp/ipython-input-2450227530.py:4: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  [test_interactions, train_val_interactions.filter(pl.col('user_id').is_in(test_users))]


130127

In [91]:
# Split data into train and validation subsets
TRAIN_USERS_FRACTION = 0.9
TRAIN_USERS = (
    train_val_interactions.get_column('user_id')
    .unique()
    .sort()
    .sample(fraction=TRAIN_USERS_FRACTION, seed=42)
)

condition = pl.col('user_id').is_in(TRAIN_USERS)

train_interactions = train_val_interactions.filter(condition)
val_interactions = train_val_interactions.filter(~condition)

len(train_interactions), len(val_interactions)

/tmp/ipython-input-1666146712.py:12: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  train_interactions = train_val_interactions.filter(condition)
/tmp/ipython-input-1666146712.py:13: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  val_interactions = train_val_interactions.filter(~condition)


(202582, 22343)

In [92]:
# Encode warm item IDs
WARM_ITEMS = train_interactions.get_column('item_id').unique().sort()
print(f'{len(WARM_ITEMS) = }')

# Keep zero for padding
item2index_warm = {item: index + 1 for index, item in enumerate(WARM_ITEMS)}

train_interactions_1 = train_interactions.with_columns(
    pl.col('item_id').replace_strict(item2index_warm)
)

# Filter out cold items from validation subset
val_interactions_1 = val_interactions.filter(pl.col('item_id').is_in(WARM_ITEMS))
val_interactions_2 = val_interactions_1.with_columns(pl.col('item_id').replace_strict(item2index_warm))

len(WARM_ITEMS) = 4689


/tmp/ipython-input-1054910192.py:13: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  val_interactions_1 = val_interactions.filter(pl.col('item_id').is_in(WARM_ITEMS))


In [93]:
# Encode cold item IDs
test_interactions = test_interactions.with_columns(
    ~pl.col('item_id').is_in(WARM_ITEMS).alias('is_cold')
)

COLD_ITEMS = test_interactions.filter(pl.col('is_cold')).get_column('item_id').unique().sort()
print(f'{len(COLD_ITEMS) = }')

bias = max(item2index_warm.values()) + 1
item2index_cold = {item: index + bias for index, item in enumerate(COLD_ITEMS)}

test_interactions_1 = test_interactions.with_columns(
    pl.col('item_id').replace_strict({**item2index_warm, **item2index_cold})
)

len(COLD_ITEMS) = 680


/tmp/ipython-input-3491545942.py:2: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  test_interactions = test_interactions.with_columns(


In [94]:
# Separate ground truth from test interactions
test_interactions_2 = test_interactions_1.with_columns(
    pl.col('user_id') #'item_id'
    .cum_count(reverse=True)
    .over('user_id', order_by='timestamp')
    .alias('position')
)

condition = pl.col('position') == 1

ground_truth = test_interactions_2.filter(condition)
test_interactions_3 = test_interactions_2.filter(~condition)

In [95]:
THRESHOLD_DT

datetime.datetime(2021, 8, 10, 0, 0)

In [96]:
len(train_interactions_1), len(val_interactions_2), len(test_interactions_3), len(ground_truth)

(202582, 22343, 117317, 12810)

In [97]:
ground_truth.get_column('timestamp').min()

datetime.datetime(2021, 8, 11, 0, 0)

In [98]:
# Apply some basic checks
assert (train_interactions_1.get_column('timestamp') <= THRESHOLD_DT).all()
assert (val_interactions_2.get_column('timestamp') <= THRESHOLD_DT).all()
assert (ground_truth.get_column('timestamp') > THRESHOLD_DT).all()

In [99]:
# Save processed data
OUTPUT_DIR = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/V2'

if not os.path.exists(os.path.join(OUTPUT_DIR, 'processed')):
    os.makedirs(os.path.join(OUTPUT_DIR, 'processed'))

train_interactions_1.write_parquet(os.path.join(OUTPUT_DIR, 'processed/train_interactions.parquet'))
val_interactions_2.write_parquet(os.path.join(OUTPUT_DIR, 'processed/val_interactions.parquet'))
test_interactions_3.write_parquet(os.path.join(OUTPUT_DIR, 'processed/test_interactions.parquet'))
ground_truth.write_parquet(os.path.join(OUTPUT_DIR, 'processed/ground_truth.parquet'))

with open(os.path.join(OUTPUT_DIR, 'processed/item2index_warm.pkl'), mode='wb') as file:
    pickle.dump(item2index_warm, file)

with open(os.path.join(OUTPUT_DIR, 'processed/item2index_cold.pkl'), mode='wb') as file:
    pickle.dump(item2index_cold, file)

In [107]:
import polars as pl


# metadata = items.with_columns(
#     [
#         pl.when(pl.col("age").is_null())
#           .then(pl.lit("null"))
#           .otherwise(pl.col("age"))
#           .alias("age"),
#         pl.when(pl.col("income").is_null())
#           .then(pl.lit("null"))
#           .otherwise(pl.col("income"))
#           .alias("income"),
#         pl.when(pl.col("sex").is_null())
#           .then(pl.lit("null"))
#           .otherwise(pl.col("sex"))
#           .alias("sex"),
#     ]
# )
metadata =  items
metadata

item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
i64,str,str,str,f64,str,str,f64,f64,str,str,str,str,str
10711,"""film""","""Поговори с ней""","""Hable con ella""",2002.0,"""драмы, зарубежные, детективы, …","""Испания""",null,16.0,null,"""Педро Альмодовар""","""Адольфо Фернандес, Ана Фернанд…","""Мелодрама легендарного Педро А…","""Поговори, ней, 2002, Испания, …"
2508,"""film""","""Голые перцы""","""Search Party""",2014.0,"""зарубежные, приключения, комед…","""США""",null,16.0,null,"""Скот Армстронг""","""Адам Палли, Брайан Хаски, Дж.Б…","""Уморительная современная комед…","""Голые, перцы, 2014, США, друзь…"
10716,"""film""","""Тактическая сила""","""Tactical Force""",2011.0,"""криминал, зарубежные, триллеры…","""Канада""",null,16.0,null,"""Адам П. Калтраро""","""Адриан Холмс, Даррен Шалави, Д…","""Профессиональный рестлер Стив …","""Тактическая, сила, 2011, Канад…"
7868,"""film""","""45 лет""","""45 Years""",2015.0,"""драмы, зарубежные, мелодрамы""","""Великобритания""",null,16.0,null,"""Эндрю Хэй""","""Александра Риддлстон-Барретт, …","""Шарлотта Рэмплинг, Том Кортни,…","""45, лет, 2015, Великобритания,…"
16268,"""film""","""Все решает мгновение""",null,1978.0,"""драмы, спорт, советские, мелод…","""СССР""",null,12.0,"""Ленфильм""","""Виктор Садовский""","""Александр Абдулов, Александр Д…","""Расчетливая чаровница из совет…","""Все, решает, мгновение, 1978, …"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
6443,"""series""","""Полярный круг""","""Arctic Circle""",2018.0,"""драмы, триллеры, криминал""","""Финляндия, Германия""",null,16.0,null,"""Ханну Салонен""","""Иина Куустонен, Максимилиан Бр…","""Во время погони за браконьерам…","""убийство, вирус, расследование…"
2367,"""series""","""Надежда""",null,2020.0,"""драмы, боевики""","""Россия""",0.0,18.0,null,"""Елена Хазанова""","""Виктория Исакова, Александр Ку…","""Оригинальный киносериал от соз…","""Надежда, 2020, Россия"""
10632,"""series""","""Сговор""","""Hassel""",2017.0,"""драмы, триллеры, криминал""","""Россия""",0.0,18.0,null,"""Эшреф Рейбрук, Амир Камдин, Эр…","""Ола Рапас, Алиетт Офейм, Уильм…","""Криминальная драма по мотивам …","""Сговор, 2017, Россия"""


In [37]:
# metadata = metadata.with_columns(
#     title=pl.concat_str(
#         [pl.col("age"), pl.col("income"), pl.col("sex")],
#         separator=","
#     )
# )


In [108]:
# Process metadata
metadata = (
    metadata
    .filter(pl.col('item_id').is_in(pl.concat((WARM_ITEMS, COLD_ITEMS))))
    .with_columns(pl.col('item_id').replace_strict({**item2index_warm, **item2index_cold}))
    .sort('item_id')
)
print(f'{metadata.shape = }')
metadata.head(1)

metadata.shape = (5369, 14)


/tmp/ipython-input-529826720.py:4: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  .filter(pl.col('item_id').is_in(pl.concat((WARM_ITEMS, COLD_ITEMS))))


item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
i64,str,str,str,f64,str,str,f64,f64,str,str,str,str,str
1,"""series""","""Смешарики: Азбука безопасности""",null,2006.0,"""русские, для детей, сериалы, х…","""Россия""",null,0.0,null,"""Алексей Горбунов""","""Антон Виноградов, Вадим Бочано…","""Опасности подстерегают на кажд…","""Смешарики, Азбука, безопасност…"


In [109]:
assert len(metadata) == len(item2index_warm) + len(item2index_cold)

In [110]:
metadata

item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
i64,str,str,str,f64,str,str,f64,f64,str,str,str,str,str
1,"""series""","""Смешарики: Азбука безопасности""",null,2006.0,"""русские, для детей, сериалы, х…","""Россия""",null,0.0,null,"""Алексей Горбунов""","""Антон Виноградов, Вадим Бочано…","""Опасности подстерегают на кажд…","""Смешарики, Азбука, безопасност…"
2,"""film""","""Ворона-проказница""","""Der kleine Rabe Socke - Das gr…",2012.0,"""мультфильм, приключения, комед…","""Германия""",null,0.0,null,"""Уте фон Мюнхов-Поль""","""Катарина Тальбах, Анна Тальбах…","""Зок - черная ворона с желтым к…","""бобер, друзья, дамба, женщина-…"
3,"""series""","""Жестокий Стамбул""","""Ruthless City""",2019.0,"""драмы""","""Турция""",null,16.0,null,"""Джевдет Мерджан, Шенол Сёнмез""","""Дениз Угур, Озан Долунай, Сера…","""Семья Агаха Караджая могуществ…",null
4,"""film""","""Мизерере""","""La marque des anges - Miserere""",2013.0,"""триллеры, криминал, детективы""","""Франция, Бельгия, Германия""",null,16.0,null,"""Сильвен Уайт""","""Марта Келлер, Тьерри Лермитт, …","""В армянском соборе Святого Иоа…","""бывший нацист, глухой ребенок,…"
5,"""series""","""Перебежчик""","""Turncoat""",2020.0,"""драмы, историческое, военные""","""Германия""",null,16.0,null,"""Флориан Галленбергер""","""Яннис Нивёнер, Себастьян Урзен…","""Вальтер Проска возвращается из…",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
5365,"""film""","""Мой ужин с Эрве""","""My Dinner with Herve""",2018.0,"""биография""","""США""",null,16.0,"""HBO""","""Саша Джерваси""","""Джейми Дорнан, Питер Динклэйдж…","""Звезда «Игры престолов» Питер …","""Мой, ужин, Эрве, 2018, США"""
5366,"""series""","""Быстрые и громкие""","""Fast N' Loud""",2016.0,"""научно-популярные""","""США""",null,12.0,null,null,"""Ричард Роулингс, Аарон Кауфман…","""Наших героев ждет суровое испы…",null
5367,"""film""","""Мама""",null,2019.0,"""драмы""","""Россия""",null,12.0,null,"""Владимир Котт""","""Линда Вахер, Отт Райдметс, Арс…","""По узорчатой от мороза глади в…","""фестиваль уличного кино, 2019,…"


In [111]:
CHECKPOINT = 'intfloat/e5-base-v2'

model = SentenceTransformer(CHECKPOINT)

item_embeddings = model.encode(
    metadata.get_column('title').to_list(),
    batch_size=512,
    show_progress_bar=True,
    normalize_embeddings=False,
)
item_embeddings.shape

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

(5369, 768)

In [113]:
warm_item_embeddings = item_embeddings[: len(WARM_ITEMS)]
cold_item_embeddings = item_embeddings[len(WARM_ITEMS) :]

In [114]:
# Save item embeddings
if not os.path.exists(os.path.join(OUTPUT_DIR, 'item_embeddings')):
    os.makedirs(os.path.join(OUTPUT_DIR, 'item_embeddings'))

np.save(os.path.join(OUTPUT_DIR, 'item_embeddings/embeddings_warm.npy'), warm_item_embeddings)
np.save(os.path.join(OUTPUT_DIR, 'item_embeddings/embeddings_cold.npy'), cold_item_embeddings)

In [ ]:
#IndexError: select(): index 11237 out of range for tensor of size [11166, 128] at dimension 0

In [127]:
train_interactions_1['user_id'].unique().shape,val_interactions_2['user_id'].unique().shape,test_interactions_3['user_id'].unique().shape,ground_truth['user_id'].unique().shape

((19870,), (2208,), (9261,), (12810,))

In [134]:
import polars as pl
import numpy as np
import pickle

path_interaction = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/interactions.parquet'
path_users = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/users.parquet'
path_items = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/items.parquet'

path_embeddings_warm = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/item_embeddings/embeddings_warm.npy'
path_embeddings_cold = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/item_embeddings/embeddings_cold.npy'

path_train  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/train_interactions.parquet'
path_val  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/val_interactions.parquet'
path_test  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/test_interactions.parquet'
path_groud  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/ground_truth.parquet'

inter_1= pl.read_parquet(path_interaction)
users_1 = pl.read_parquet(path_users)
items_1 = pl.read_parquet(path_items)
emb_warm = np.load(path_embeddings_warm)
emb_cold = np.load(path_embeddings_cold)

train_1= pl.read_parquet(path_train)
val_1= pl.read_parquet(path_val)
test_1= pl.read_parquet(path_test)
groud= pl.read_parquet(path_groud)


with open('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/item2index_warm.pkl', 'rb') as f:
    item2index_warm_1 = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Данные для обучения/processed/item2index_cold.pkl', 'rb') as f:
    item2index_cold_1 = pickle.load(f)




In [135]:
emb_warm.shape, emb_cold.shape,len(item2index_warm_1),len(item2index_cold_1)

((22066, 768), (6735, 768), 22066, 6735)

In [ ]:
train_1['user_id'].unique().shape,val_1['user_id'].unique().shape,test_1['user_id'].unique().shape,groud['user_id'].unique().shape

((22066,), (14136,), (9261,), (12810,))

In [ ]:
len(inter_1['user_id'].unique())

50000

In [ ]:
len(item2index_warm_1)

22066

In [ ]:
train_1['user_id'].unique().shape,len(item2index_warm_1),emb_warm.shape,test_interactions_3.filter(pl.col('is_cold')==True)['user_id'].unique().shape

((22066,), 22066, (22066, 768), (3186,))

In [ ]:
# Сортировка по убыванию
train_1_sorted = train_1.sort('user_id', descending=True)

train_1_sorted

user_id,item_id,rating,timestamp
i64,i64,f64,datetime[ns]
22066,3076,1.0,2021-06-23 00:00:00
22066,12396,0.0,2021-06-24 00:00:00
22066,12995,4.0,2021-06-24 00:00:00
22065,11640,100.0,2021-06-16 00:00:00
22065,13865,100.0,2021-06-21 00:00:00
…,…,…,…
2,13865,11.0,2021-08-05 00:00:00
1,7102,3.0,2021-07-19 00:00:00
1,14359,2.0,2021-07-19 00:00:00


In [ ]:
for i,v in item2index_warm.items():

  if v ==22065:
    print(i,v)

    break

1097458 22065


In [ ]:
train_interactions.filter(pl.col('user_id')==1097458)

user_id,item_id,rating,timestamp
i64,i64,f64,datetime[ns]
1097458,11640,100.0,2021-06-16 00:00:00
1097458,13865,100.0,2021-06-21 00:00:00
1097458,15297,88.0,2021-06-28 00:00:00


In [ ]:
groud['user_id'].unique()

user_id
i64
3
13
15
18
23
…
28797
28798
28799


In [ ]:
groud.filter(pl.col('is_cold')==True)['user_id'].unique()

user_id
i64
22067
22068
22069
22070
22071
…
28797
28798
28799


In [ ]:
val_interactions_2['user_id'].unique()

user_id
i64
1
3
4
5
7
…
22054
22055
22056


In [ ]:
emb_cold.shape,len(item2index_cold_1),groud.filter(pl.col('is_cold')==True)['user_id'].unique().shape

((6735, 768), 6735, (6735,))

In [ ]:
# Сортировка по убыванию
groud_sorted = groud.sort('user_id', descending=True)

groud_sorted

user_id,item_id,rating,timestamp,is_cold,position
i64,i64,f64,datetime[ns],bool,u32
28801,16484,2.0,2021-08-22 00:00:00,true,1
28800,13865,1.0,2021-08-20 00:00:00,true,1
28799,6443,0.0,2021-08-21 00:00:00,true,1
28798,1131,56.0,2021-08-13 00:00:00,true,1
28797,14317,100.0,2021-08-19 00:00:00,true,1
…,…,…,…,…,…
23,13615,48.0,2021-08-18 00:00:00,false,1
18,6006,0.0,2021-08-22 00:00:00,false,1
15,5569,0.0,2021-08-21 00:00:00,false,1


In [ ]:
for i,v in item2index_cold_1.items():

  if v ==28797:
    print(i,v)

    break

1096427 28797


In [43]:
train_1['user_id'].unique()

user_id
i64
1
2
3
4
5
…
22062
22063
22064


In [47]:
train_1['user_id'].unique().shape,len(item2index_warm_1),emb_warm.shape,test_interactions_3.filter(pl.col('is_cold')==False)['user_id'].unique().shape

((22066,), 22066, (22066, 768), (6075,))

In [52]:
test_interactions_3['user_id'].unique().shape,len(item2index_cold_1),emb_cold.shape,groud.filter(pl.col('is_cold')==True)['user_id'].unique().shape

((9261,), 6735, (6735, 768), (6735,))

In [61]:
groud.filter(pl.col('is_cold')==True)['user_id'].unique().shape[0]+groud.filter(pl.col('is_cold')==False)['user_id'].unique().shape[0],test_interactions_3['user_id'].unique().shape

(12810, (9261,))

In [62]:
test_interactions_3.filter(pl.col('is_cold')==True)['user_id'].unique().shape[0]+test_interactions_3.filter(pl.col('is_cold')==False)['user_id'].unique().shape[0],

(9261,)

In [115]:
import polars as pl
import numpy as np
import pickle

path_interaction = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/interactions.parquet'
path_users = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/users.parquet'
path_items = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/kion_sars_rec_it_bias/Parquet_data_u_i_interactions/items.parquet'

path_embeddings_warm = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/item_embeddings/embeddings_warm.npy'
path_embeddings_cold = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/item_embeddings/embeddings_cold.npy'

path_train  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/train_interactions.parquet'
path_val  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/val_interactions.parquet'
path_test  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/test_interactions.parquet'
path_groud  = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/ground_truth.parquet'

inter_1= pl.read_parquet(path_interaction)
users_1 = pl.read_parquet(path_users)
items_1 = pl.read_parquet(path_items)
emb_warm = np.load(path_embeddings_warm)
emb_cold = np.load(path_embeddings_cold)

train_1= pl.read_parquet(path_train)
val_1= pl.read_parquet(path_val)
test_1= pl.read_parquet(path_test)
groud= pl.read_parquet(path_groud)


with open('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/item2index_warm.pkl', 'rb') as f:
    item2index_warm_1 = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/data/beauty/processed/item2index_cold.pkl', 'rb') as f:
    item2index_cold_1 = pickle.load(f)


In [118]:
emb_warm.shape

(11165, 768)

In [119]:
emb_cold.shape

(568, 768)

In [125]:
train_1['user_id'].unique().shape,val_1['user_id'].unique().shape,test_1['user_id'].unique().shape,groud['user_id'].unique().shape

((17393,), (1933,), (4867,), (4867,))

In [128]:
train_interactions_1['user_id'].unique().shape,val_interactions_2['user_id'].unique().shape,test_interactions_3['user_id'].unique().shape,ground_truth['user_id'].unique().shape

((19870,), (2208,), (9261,), (12810,))

In [133]:
warm_item_embeddings.shape,cold_item_embeddings.shape,len(item2index_warm_1),len(item2index_cold_1)

((4689, 768), (680, 768), 11165, 568)